<a href="https://colab.research.google.com/github/nh0875/Desafios-Procesamiento-del-Habla-974/blob/main/Desafio_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tony "El chatbot coach" (Chatbot con TFIDF/Similitud del coseno)

Voy a utilizar un chatbot con un motor de busqueda por similitud, ya que mi idea es que el chatbot sirva para responder preguntas comunes sobre que acciones tomar en un partido de basquet orientado a personas que no saben jugarlo o que estan comenzando a hacerlo, que hacer en situaciones especificas como tambien situaciones generales de un partido, voy a ayudarme de Gemini para poder abarcar a la mayoria de posibles preguntas que podrian llegarle al bot, como tambien las respuestas que deberia devolver mi bot, se va a llamar **"Tony"**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

preguntas = [
    "¿Cuál es la diferencia entre una falta personal y una falta técnica?",
    "¿Qué es una violación de los tres segundos y cómo se aplica en la zona restringida?",
    "¿Cuál es la diferencia entre un bloqueo y una carga?",
    "¿Qué es una cortina (o pantalla) y cómo se utiliza en el juego ofensivo?",
    "¿Qué es un paso y cómo se sanciona?",
    "¿Qué es un doble dribling y cómo se evita?",
    "¿Cuál es la diferencia entre un tiro libre y un tiro/saque de línea de fondo?",
    "¿Qué es un cambio de defensa y cuáles son los tipos más comunes?",
    "¿Qué es una violación de 24 segundos y cómo se aplica en el juego?",
    "¿Cuál es la importancia de la rotación de balón en el juego ofensivo?",
    "Si mi equipo está perdiendo por muchos puntos al final del partido, ¿qué debería hacer?",
    "Si tengo la pelota y estoy siendo marcado por un defensor muy alto, ¿qué puedo hacer?",
    "Si un compañero mío tira de tres puntos y falla, ¿qué debería hacer?",
    "Si estoy defendiendo a un jugador mucho más rápido que yo, ¿qué estrategia puedo usar?",
    "Si me encuentro solo debajo del aro, ¿qué tipo de tiro debo intentar?",
    "¿Cómo puedo mejorar mi tiro?",
    "¿Qué puedo hacer para mejorar mi manejo de balón?",
    "¿Cómo puedo mejorar mi defensa?",
    "¿Qué es un pick-and-roll y cómo puedo ejecutarlo?",
    "¿Qué es una fast break y cómo puedo participar en ella?",
    "¿Cómo puedo mejorar mi juego en equipo?",
    "¿Qué es una zona de presión y cómo puedo jugar contra ella?",
    "¿Cómo puedo mantenerme motivado para seguir practicando básquetbol?"
]

respuestas = [
    "Una falta personal se comete contra un jugador y suele ser un contacto físico. Una falta técnica se sanciona por acciones antideportivas o protestas hacia los árbitros.",
    "Un jugador ofensivo no puede permanecer más de tres segundos dentro de la zona restringida del equipo contrario sin botar el balón o intentar un tiro.",
    "Un bloqueo es una acción legal en la que un defensor impide el avance de un atacante sin cometer una falta. Una carga ocurre cuando el defensor se mueve hacia el atacante, impidiendo su progreso de manera ilegal.",
    "Una cortina es una acción en la que un jugador se posiciona cerca de un defensor para obstruir su camino y permitir que un compañero de equipo reciba un pase sin marca.",
    "Un paso es una violación que ocurre cuando un jugador da más de un paso con el balón en las manos sin botarlo.",
    "Un doble dribling ocurre cuando un jugador detiene el drible y luego lo vuelve a iniciar con la misma mano. Para evitarlo, es importante mantener el balón en constante movimiento o pasarlo a un compañero.",
    "Un tiro libre se otorga como sanción por una falta y se lanza desde la línea de tiros libres. Un tiro/saque de línea de fondo se realiza cuando la pelota sale fuera de límites por la línea de fondo y el equipo contrario tiene la posesión.",
    "Un cambio de defensa es una modificación en la estrategia defensiva de un equipo. Los tipos más comunes son la defensa hombre a hombre y la defensa zonal.",
    "Un equipo tiene 24 segundos para realizar un tiro a canasta una vez que tiene la posesión del balón. Si el tiempo se agota sin que se realice un tiro, se produce una violación y el balón pasa al equipo contrario.",
    "La rotación de balón es esencial para mantener a la defensa desorganizada, crear oportunidades de tiro abiertas y evitar que un solo jugador se cargue con la responsabilidad de anotar.",
    "En esta situación, es importante mantener la calma y seguir jugando como equipo. Puedes intentar forzar robos de balón, buscar tiros de tres puntos para acortar la distancia y evitar cometer faltas innecesarias.",
    "Puedes intentar un tiro, pasar la pelota a un compañero libre, o utilizar un movimiento falso para crear espacio y encestar.",
    "Deberías estar listo para el rebote, ya que es probable que la pelota rebote hacia el área. Si la obtienes, puedes intentar un nuevo tiro o pasarla a un compañero mejor posicionado.",
    "Mantén una distancia adecuada, utiliza tus brazos para evitar que drible fácilmente y trata de anticipar sus movimientos.",
    "Si estás solo bajo el aro, lo ideal es intentar un tiro fácil, como una bandeja o un gancho.",
    "Para mejorar tu tiro, practica regularmente, enfócate en la técnica correcta y trabaja en tu fuerza superior del cuerpo.",
    "Practica driblar con ambas manos, a diferentes velocidades y en espacios reducidos. También puedes realizar ejercicios de coordinación con la pelota.",
    "Para mejorar tu defensa, trabaja en tu agilidad, fuerza y capacidad de anticipar los movimientos de tus oponentes.",
    "Un pick-and-roll es una jugada en la que un jugador establece una pantalla (pick) para un compañero que está driblando, con el objetivo de crear una ventaja numérica.",
    "Un fast break es un ataque rápido después de un rebote defensivo o una intercepción. Para participar en un fast break, debes correr rápidamente hacia la canasta contraria y buscar un pase para finalizar la jugada.",
    "La comunicación, la confianza en tus compañeros y la disposición a compartir la pelota son fundamentales para un buen juego en equipo.",
    "Una zona de presión es una defensa en la que los jugadores se distribuyen en zonas del campo en lugar de marcar a un jugador específico. Para jugar contra una zona, es importante mover el balón rápidamente y buscar tiros abiertos.",
    "Establece metas realistas, encuentra compañeros de entrenamiento, únete a una liga y celebra tus logros."
]

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(preguntas)

def responder(consut_):
    consulta_vec = vectorizer.transform([consut_])
    similitudes = cosine_similarity(consulta_vec, tfidf_matrix).flatten()
    indice_mejor_coincidencia = similitudes.argmax()
    return respuestas[indice_mejor_coincidencia]


In [ ]:
consulta = str(input("Escribe tu consulta sobre el basquet: "))
responder(consulta)

Escribe tu consulta sobre el basquet: que es un tiro libre


'Un tiro libre se otorga como sanción por una falta y se lanza desde la línea de tiros libres. Un tiro de línea de fondo se realiza cuando la pelota sale fuera de límites por la línea de fondo y el equipo contrario tiene la posesión.'

# Categorizador de Reseñas (Chatbot con Embeddings, **Modelo FastText** )

Es un bot el cual clasifica reseñas de restaurantes basandose en palabras clave para determinar si la reseña es positiva o negativa



In [ ]:
!pip install fasttext
!pip install spacy
!python -m spacy download en_core_web_sm

In [25]:
import fasttext
import spacy

modelo_ft=fasttext.load_model('/content/cc.en.300.bin')
nlp=spacy.load("en_core_web_sm")
def preprocesar_texto(texto):
    doc = nlp(texto)
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

def cargar_opiniones(archivo):
    with open(archivo, 'r', encoding='utf-8') as f:
        opiniones=[line.strip() for line in f if line.strip()]
    return opiniones

def clasificar_opinion(opinion):
    opinion_procesada = preprocesar_texto(opinion)
    embedding_opinion = modelo_ft.get_sentence_vector(opinion_procesada)

    if 'good' in opinion_procesada or 'excellent' in opinion_procesada:
        return "Positive"
    else:
        return "Negative"

def chatbot_opiniones(archivo_opiniones):
    opiniones=cargar_opiniones(archivo_opiniones)
    for opinion in opiniones:
        clasificación=clasificar_opinion(opinion)
        print(f"Opinión: {opinion.strip()}")
        print(f"Clasificación: {clasificación}\n")

archivo='/content/Opiniones_restaurant.txt'
chatbot_opiniones(archivo)

Opinión: "I absolutely loved the atmosphere at The Gourmet Garden. It felt cozy and welcoming, perfect for a date night!"
Clasificación: Negative

Opinión: "The service was bad slow during my last visit. We waited over 30 minutes for our drinks to arrive, which was frustrating."
Clasificación: Negative

Opinión: "The menu offers a good variety of dishes. I tried the seafood risotto, and it was rich and flavorful. Definitely coming back for more!"
Clasificación: Positive

Opinión: "I was disappointed with my meal. The chicken was dry, and I expected more seasoning for the price I paid."
Clasificación: Negative

Opinión: "The staff was friendly and excellent, making sure we had everything we needed. The dessert selection was heavenly, especially the tiramisu!"
Clasificación: Positive

Opinión: "I went there based on the reviews, but my experience was not great. The pasta was undercooked, and the sauce tasted bland."
Clasificación: Negative

Opinión: "What a delightful experience! The Gou